In [1]:
import sys

import tensorflow as tf
import numpy as np

sys.path.append('E:/xinlib')
import xin
import chaos

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
root = 'E:/Data/datasets/flower_photos/'
X, Y = xin.Path2Dataset(root).path2data(True, 28)

data_ = X.concatenate()
data = xin.norm(data_)
data_mat = chaos.TensorShape(data).tensor_to_matrix()
labels = chaos.TensorShape(Y.codes).tensor_to_matrix()
data_mat_shape = chaos.TensorShape(data_mat).get_shape()

# 构造一个线性模型
n_class = len(Y.categories)
n_samples, n_features = data_mat_shape
shape = [n_features, n_class]
b = tf.Variable(tf.zeros([n_samples, 1]))
W = tf.Variable(tf.random_normal(shape))
output = tf.matmul(data_mat, W) + b
y = chaos.TensorShape(output).softmax()

In [76]:
y_ = chaos.TensorShape(xin.one_hot(Y.codes)[0]).tensor_to_matrix()
y_ 

<tf.Tensor 'Reshape_4:0' shape=(3670, 5) dtype=float32>

In [77]:
y

<tf.Tensor 'truediv:0' shape=(3670, 5) dtype=float32>

In [78]:
chaos.tensor_to_array(tf.log(y)* y_)

array([[-0.       , -0.       , -1.559558 , -0.       , -0.       ],
       [-2.1952057, -0.       , -0.       , -0.       , -0.       ],
       [-0.       , -1.5968845, -0.       , -0.       , -0.       ],
       ...,
       [-1.8179803, -0.       , -0.       , -0.       , -0.       ],
       [-0.       , -0.       , -0.       , -0.       , -1.2865427],
       [-0.       , -0.       , -0.       , -2.7867048, -0.       ]],
      dtype=float32)

In [81]:
# 最小化方差
loss = tf.reduce_mean(tf.reduce_sum(tf.log(y) * y_, axis=1))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train_op =  optimizer.minimize(loss)

In [82]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for step in range(100):
        sess.run(train_op)
        if step % 20 == 0:
            print(step, sess.run(W), sess.run(b))

0 [[-1.223632   -1.2613049  -0.24638188 -0.49709928 -1.1250569 ]
 [-0.10357273 -1.1396432   0.63435966  0.42433146 -0.57293725]
 [-1.6094025   0.19016889  0.4452387  -1.6781834  -0.64108187]
 ...
 [-1.3449205   2.0138128   0.43344054 -1.2696772   0.6501979 ]
 [ 1.5105501   0.09992945 -1.0699633   0.03044954 -0.8973119 ]
 [-2.1030834   1.7142879   0.24506794  0.07254819 -0.24519417]] [[-2.1827873e-11]
 [ 7.2759576e-12]
 [-7.2759576e-12]
 ...
 [ 2.1827873e-11]
 [ 0.0000000e+00]
 [-3.6379788e-12]]
20 [[-1.2260745  -1.259217   -0.23329271 -0.5060982  -1.1287923 ]
 [-0.10721608 -1.139508    0.6519596   0.4119699  -0.57466745]
 [-1.6118249   0.1932179   0.45956388 -1.6903485  -0.6438688 ]
 ...
 [-1.347749    2.0125854   0.44436085 -1.2751355   0.6487914 ]
 [ 1.505622    0.09588666 -1.0527365   0.0230014  -0.8981197 ]
 [-2.1083698   1.7126043   0.25619966  0.06994473 -0.24675235]] [[-4.3655746e-11]
 [ 7.0940587e-11]
 [-7.2759576e-12]
 ...
 [-1.4551915e-11]
 [-5.8207661e-11]
 [ 6.5483619e-11]]

In [90]:
acc = tf.reduce_mean(tf.reduce_sum(y*y_, axis=1))
chaos.tensor_to_array(acc)

0.20329416

In [85]:
y

<tf.Tensor 'truediv:0' shape=(3670, 5) dtype=float32>

In [86]:
y_

<tf.Tensor 'Reshape_4:0' shape=(3670, 5) dtype=float32>